# Exercise Fourteen: Project Design Starter
In this exercise, you'll be planning out a complex project. You'll draw in some code, but focus on commenting to describe your project structure. The sample document below will guide you through organizing and annotating your project design. The primary components you'll include are:

- Dependencies: What modules will your project need?
- Collection: Where is your data coming from?
- Processing: How will you format and process your data?
- Analysis: What techniques will you use to understand your data?
- Visualization: How will you visualize and explore your data?

# Project Overview
I plan on extracting information from Twitter using #BlerDcon. BlerDcon is a convention — the event’s name signals that it specifically caters to blerds (black nerds), BlerDCon is an inclusive space for all POC, LGBTQ+, female, and disabled fans. Pulling data from the 2021 convention, I seek to gather more information and insight into the topics dicussed around this venue and the sentiments of the participants. 

# Dependencies
Importing modules to extract data from hastags on twitter, compile that information into a digestible list, and then use that info to create visualizations for the data.

In [50]:
# Importing Config to pull credentials for Twitter API
import configparser

#Importing Twitter API
import tweepy

import datetime

import csv

#Importing to help extract data
import requests
from bs4 import BeautifulSoup

#Importing to map out geographical locations of responses
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

#importing to plot findings in visualization
import matplotlib.pyplot as plt

#importing for map visualization
import cartopy.crs as ccrs
from matplotlib.patches import Circle

#importing to assist in compliling 
import re

#Importing for punctuation
from nltk.tokenize import WordPunctTokenizer

#importing to make a wordcloud for visualizations
from wordcloud import WordCloud, STOPWORDS

#To turn work cloud into an image
import numpy as np
from PIL import Image
import random

#Importing to gather sentiments from comments
import nltk.data
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import sentiment
from nltk import word_tokenize

# Importing Pandas to handle collected Twitter data 
import pandas as pd

import os

# Collection
Collecting data from recent 2021 convention hastag #BlerDcon. Starting date from pull will begin 01-01-2021 to current time. This can be changed later to look into other year conventions. 

In [52]:
#Pulling credentials file, so Twiiter Developer account can be accessed
CONFIG = configparser.ConfigParser()
CONFIG.read('credentials.ini')

#Tweepy
auth = tweepy.OAuthHandler(CONFIG['DEFAULT']['consumer_key'], CONFIG['DEFAULT']['consumer_secret'])
auth.set_access_token(CONFIG['DEFAULT']['access_token'], CONFIG['DEFAULT']['access_token_secret'])

api = tweepy.API(auth, wait_on_rate_limit=True)

public_tweets = api.home_timeline()
for tweet in public_tweets:
    #dont need to see everything
    print(tweet.text [0:10])

i’m always
I want the
The Bipart
It’s like 
I got a fe
RT @blushi
Hi. So gla
A New Jers
Whole time
Niggas wil
A lot of o
What Does 
“A man to 
There are 
Things evo
Missing Ar
The U.S. m
RT @UCF_Fo
RT @tminsb
About me: 


In [54]:
#pulling info in from specific hashtag
search_words = "#BlerDCon2021"
date_since = "2020-01-01"
tweets = tweepy.Cursor(api.search_tweets,
              q=search_words, lang="en").items(300)

In [57]:
output = []
for tweet in tweets:
    text = tweet._json
    print(text)
    favourite_count = tweet.favorite_count
    retweet_count = tweet.retweet_count
    created_at = tweet.created_at
    
    line = {'text' : text, 'favourite_count' : favourite_count, 'retweet_count' : retweet_count, 'created_at' : created_at}
    output.append(line)

# Processing
After your data has been collected or imported, store it in a format that works for your purposes. This can vary: for Twitter analysis, it might be a Pandas dataframe, while for text, you might build a document term matrix.

In [ ]:
'''#using pandas to sort tweets
tweets_sorted = [[tweet.user.screen_name, tweet.geo, tweet.user.location, tweet.text] for tweet in tweets]

#caluculating coordinates of Tweets -- doesnt look like too many locations (may not use for data set)
tdf = pd.DataFrame(data=tweets_sorted, columns=['user', 'coordinates','location', 'tweet'])
print(tdf)

#may need to look for word count instead'''

In [ ]:
'''BlerDcon_topics = pd.DataFrame(review_dict)
print(BlerDcon_topics)'''

# Analysis
Think across all of the methods we've tried this semester - what combination would be most helpful for your goals? Include code sections for each method you think is important. In most cases, a combination will be most revealing: for instance, you might employ several different textual analysis frameworks on a set of documents. Use at least two distinctly different methods of analysis.

# Visualization
Finally, think about the visualizations that would be most useful to sharing and exploring your data. Consider both static and dynamic approaches from the different libraries we've worked with this semester. Include at least two preliminary visualizations.